# <center>detritalPy-nmf</center>
### <center>Glenn R. Sharman</center>
#### <center>Code for conducting 'bottom-up', or inverse, mixture modeling of detrital zircon U-Pb age distributions</center>

#### <center>Version 0.1 (Beta)</center>
##### <center>Updated Feb 9, 2024</center>

<center>This is a <b>Beta</b> release detritalPy-nmf. Please contact Glenn Sharman (gsharman@uark.edu) if you encounter any errors or problems.</center>

<center>If you find this code helpful in your research, please consider citing Sharman and Johnstone (2017): Earth and Planetary Science Letters (https://doi.org/10.1016/j.epsl.2017.07.044) and/or Saylor et al. (2019): Earth and Planetary Science Letters (https://doi.org/10.1016/j.epsl.2019.01.044)</center>

### I. Import required modules
This step must by run initially, but then does not need to be run again for the remainder of the analysis session.

In [ ]:
import detritalpy
import detritalpy.detritalFuncs as dFunc
import detritalpy.nmfFuncs as nmf
import pathlib
import matplotlib
import time
from operator import itemgetter
import pandas as pd
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format = 'retina' # For improving matplotlib figure resolution
matplotlib.rcParams['pdf.fonttype'] = 42 # For allowing preservation of fonts upon importing into Adobe Illustrator
matplotlib.rcParams['ps.fonttype'] = 42
print('detritalPy version: ',detritalpy.__version__)

### II. Import the dataset as an Excel file and select samples to analyze
This step must by run initially, and should be repeated if any changes are made to the dataset in Excel

<b><i>Pro tip:</b></i> If you have many samples to include in the analysis, it is tediuos to type them out manually. You can alternatively select all the samples in the dataset using the example syntax below, or you can check out the "detritalPy concatenator.xlsx" on https://github.com/grsharman/detritalPy for a fast way of formatting many samples for analysis

<b>Example 1: </b>This is the Grand Canyon dataset presented by Saylor et al. (2019): Earth and Planetary Science Letters (https://doi.org/10.1016/j.epsl.2019.01.044). Please refer to this publication for additional information.

In [ ]:
dataToLoad = ['example-data/Grand_Canyon_Data-dPy_input.xlsx']

main_df, main_byid_df, samples_df, analyses_df = dFunc.loadDataExcel(dataToLoad, dataSheet ='U_Th_Pb')

sampleList = main_byid_df['Sample_ID'].tolist() # Select individual samples (all of them)

sigma = '1sigma' # Enter '1sigma' if your database has 1-sigma uncertainties. Otherwise enter '2sigma'.

ages, errors, numGrains, labels = dFunc.sampleToData(sampleList, main_byid_df, sigma = sigma);

<b>Example 2:</b> This is the Book Cliffs dataset presented by Saylor et al. (2019): Earth and Planetary Science Letters (https://doi.org/10.1016/j.epsl.2019.01.044). Please refer to this publication for additional information.

In [ ]:
dataToLoad = ['example-data/Book_Cliffs_Data-dPy_input.xlsx']

main_df, main_byid_df, samples_df, analyses_df = dFunc.loadDataExcel(dataToLoad, dataSheet ='U_Th_Pb')

sampleList = main_byid_df['Sample_ID'].tolist() # Select individual samples (all of them)

sigma = '1sigma' # Enter '1sigma' if your database has 1-sigma uncertainties. Otherwise enter '2sigma'.

ages, errors, numGrains, labels = dFunc.sampleToData(sampleList, main_byid_df, sigma = sigma);

Optional: run the cell below to plot a distribution of sample size (number of analyses per sample) in the dataset

In [ ]:
dFunc.plotSampleDist(main_byid_df, numBins=25)

## Plot detrital age distributions
NMF is based on relative age distributions (i.e., KDE or PDP). Your results will depend on the type of age distribution that you calculate. <b><i>Important:</b></i> Plotting your data is an important step in using NMF because you will use these distributions as inputs to the algorithm.

In [ ]:
# Enter plot options below
whatToPlot = 'relative' # Options: cumulative, relative, or both
separateSubplots = False # Set to True to plot each relative age distribution in a separate subplot (allows histogram and pie)

# Specify the age range (Myr) that you want to plot
x1 = 0
x2 = 4000
plotLog = False # Set to True to plot the x-axis as a log scale

# Specify the plot dimensions
w = 10 # width of the plot
c = 4 # height of CDF panel
h = 10 # height of the relative panel (only required if separateSubplots is False). Options: 'auto' or an integer

xdif = 1 # Specify the interval (Myr) over which distributions are calculated

# Cumulative distribution options
plotCDF = False # Plot the CDF discretized at xdif interval
plotCPDP = False # Plot the cumulative PDP
plotCKDE = False # Plot the cumulative KDE
plotDKW = False # Plot the 95% confidence interval of the CDF (Dvoretsky-Kiefer-Wolfowitz inequality)

# Relative distribution options
normPlots = False # Will normalize the PDP/KDE if yes to True (if separateSubplots is True)

plotKDE = True # Set to True if want to plot KDE
colorKDE = False # Will color KDE according to same coloration as used in CDF plotting
colorKDEbyAge = False # Will color KDE according to age populations if set to True
bw = 15 # Specify the KDE bandwidth. Options are 'optimizedFixed', 'optimizedVariable', or a number (bandwidth in Myr). Multiple bandwidths can be entered in a list if using a split KDE
bw_x = None # Specify x-axis split location (if using a split KDE) (e.g., [300])

plotPDP = False # Set to True if want to plot PDP
colorPDP = False # Will color PDP according to same coloration as used in CDF plotting
colorPDPbyAge = False # Will color PDP according to age populations if set to True

plotColorBar = False # Color age categories as vertical bars, can add white bars to create blank space between other colored bars

plotHist = False # Set to True to plot a histogram (only available when separateSubplots is True)
b = 5 # Specify the histogram bin size (Myr)

plotPIE = False # Will plot a pie diagram (only available when separateSubplots is True)

# Specify  age categories for colored KDE, PDP, and/or pie plots
agebins = [0,300,800,1300,1550,1800,3500]
agebinsc = ['darkred','green','navy','orange','saddlebrown','darkgray']

plotAgePeaks = False # Will identify and plot age peaks
agePeakOptions = ['PDP', 0.01, 5, 2, True] # [distType, threshold, minDist, minPeakSize, labels]

fig = dFunc.plotAll(sampleList, ages, errors, numGrains, labels, whatToPlot=whatToPlot, separateSubplots=separateSubplots, plotCDF=plotCDF,
                    plotCPDP=plotCPDP, plotCKDE=plotCKDE, plotDKW=plotDKW, normPlots=normPlots, plotKDE=plotKDE, colorKDE=colorKDE,
                    colorKDEbyAge=colorKDEbyAge, plotPDP=plotPDP, colorPDP=colorPDP, colorPDPbyAge=colorPDPbyAge, plotColorBar=plotColorBar, 
                    plotHist=plotHist, plotLog=plotLog, plotPIE=plotPIE, x1=x1, x2=x2, b=b, bw=bw, xdif=xdif, agebins=agebins,
                    agebinsc=agebinsc, w=w, c=c, h=h, plotAgePeaks=plotAgePeaks, agePeakOptions=agePeakOptions, CDFlw=3, KDElw=1, PDPlw=1, bw_x=bw_x)

In [ ]:
pathlib.Path('Output').mkdir(parents=True, exist_ok=True) # Recursively creates the directory and does not raise an exception if the directory already exists 
fig.savefig('Output/dPy-nmf_input_age_distributions.pdf')

## Non-Negative Matrix Factorization (NMF)

NMF aims to solve the equation X = WH + E, where X is a matrix with the age distributions that you are modeling, W is a matrix with sample weightings, H is a matrix with end-member age distributions, and E is residual error. See Saylor et al. (2017) Earth and Planetary Science Letters (https://doi.org/10.1016/j.epsl.2019.01.044) for additional information.

In [ ]:
dist_type = 'KDE' # Options: 'KDE' or 'PDP'

nEMs = 15 # The number of end-member scenarios to consider
max_iter = 10000 # Maximum number of iterations before timing out
tol = 1e-8 # Tolerance of the stopping condition

# Execute NMF
xAge, X, Xmodeled, W, Wnorm, H, modelRecstErr, nint, in_r2_avg, in_r2_range, in_Vmax_avg, in_Vmax_range, out_r2, out_Vmax = nmf.nmf(ages=ages, errors=errors, dist_type=dist_type, bw=bw, bw_x=bw_x, nEMs=nEMs, max_iter=max_iter,
       tol=tol, verbose=True)

<b>Note:</b> NMF may struggle to differentiate end-members in datasets with a low range in inter-sample dissimilarity

## Plot the model reconstruction error as a function of # of end-member scenarios
The optimal number of end-members to use may be the point at which there is a break in slope of this plot (Saylor et al., 2019: EPSL).

In [ ]:
plot_width = 8.0
plot_height = 3.0

fig = nmf.plot_reconstr_error(modelRecstErr=modelRecstErr,
                          plot_width=plot_width, plot_height=plot_height,)

<b>Recommended exercise: </b>In some datasets, the optimal number of end-member selected is dependent on the total number of end-members you choose to run. Try modeling 10 end-members intead of 15 and see if the suggested number changes. 

In [ ]:
pathlib.Path('Output').mkdir(parents=True, exist_ok=True) # Recursively creates the directory and does not raise an exception if the directory already exists 
fig.savefig('Output/dPy-nmf_model_reconstruction_error.pdf')

## Compare modeled samples with input samples

## Plot a Summary of End-Member Age Distributions

In [ ]:
# Specify the minimum number of end-members to plot
EMs_min = 1 # Recommended value = 1 or 2

# Specify the maximum number of end-members to plot
# Note: You may use the 'optimal' number from the preceding step as a guideline
EMs_max = 6

# Specify the age range (Myr) that you want to plot
x_min = 0
x_max = 3000
plotLog = False # Set to True to plot the x-axis as a log scale

plot_width_multiplier = 1.0
plot_height_multiplier = 1.0

# Specify the plot dimensions
c = 2 # Height of CDF plot (must be integer)
w = 3.0 # Width of individual subplots (total width equals len(H)*w)

# For filling the area under the curve (if colorByAge = True)
# Sharman et al. 2015 scheme
agebins = [0, 23, 65, 85, 100, 135, 200, 300, 500, 4500]
agebinsc = ['slategray','royalblue','gold','red','darkred','purple','navy','gray','saddlebrown']

# 'Default' or specify colors as a list or as a single color (e.g., 'gray')
EMcolors = 'Default'
#EMcolors = ['red','blue','green','orange','purple','black','gray','coral','darkred','goldenrod','lavender','yellow','darkgray','lightgray','saddlebrown']

colorByAge = False
fillBetween = True

fig = nmf.plot_end_members(H=H, xAge=xAge, EMs_min=EMs_min, EMs_max=EMs_max, x_min=x_min, x_max=x_max,
                           plotLog=plotLog, plot_width_multiplier=plot_width_multiplier,
                           plot_height_multiplier=plot_height_multiplier,
                           c=c, w=w, agebins=agebins,
                          agebinsc=agebinsc, EMcolors=EMcolors, colorByAge=colorByAge,
                          fillBetween=fillBetween);


In [ ]:
pathlib.Path('Output').mkdir(parents=True, exist_ok=True) # Recursively creates the directory and does not raise an exception if the directory already exists 
fig.savefig('Output/dPy-nmf_reconstructed_sources.pdf')

## Plot end-member abundance in each sample

In [ ]:
# 'Default' or specify colors as a list or as a single color (e.g., 'gray')
EMcolors = 'Default'

# Specify the minimum number of end-members to plot
EMs_min = 1 # Recommended value = 1 or 2

# Specify the maximum number of end-members to plot
# Note: You may use the 'optimal' number from the preceding step as a guideline
EMs_max = 5

# Parameters that control figure size
plot_width_multiplier = 2.0
plot_height_multiplier = 0.25

bar_height = 0.9 # Height of bars

fig = nmf.plot_end_member_abundance(labels=labels, Wnorm=Wnorm, EMcolors=EMcolors, EMs_min=EMs_min, EMs_max=EMs_max,
                                    plot_width_multiplier=plot_width_multiplier, plot_height_multiplier=plot_height_multiplier,
                                    bar_height=bar_height);

In [ ]:
pathlib.Path('Output').mkdir(parents=True, exist_ok=True) # Recursively creates the directory and does not raise an exception if the directory already exists 
fig.savefig('Output/dPy-nmf_end-member_abundances.pdf')

## Plot end-member abundance in each sample as pie plots

In [ ]:
# 'Default' or specify colors as a list or as a single color (e.g., 'gray')
EMcolors = 'Default'

# Specify the minimum number of end-members to plot
EMs_min = 2 # Recommended value = 1 or 2

# Specify the maximum number of end-members to plot
# Note: You may use the 'optimal' number from the preceding step as a guideline
EMs_max = 5

# Parameters that control figure size
plot_width_multiplier = 2.0
plot_height_multiplier = 2.0

fig = nmf.plot_end_member_abundance_as_pie(labels=labels, Wnorm=Wnorm, EMcolors=EMcolors, EMs_min=EMs_min, EMs_max=EMs_max,
                                    plot_width_multiplier=plot_width_multiplier, plot_height_multiplier=plot_height_multiplier);

In [ ]:
pathlib.Path('Output').mkdir(parents=True, exist_ok=True) # Recursively creates the directory and does not raise an exception if the directory already exists 
fig.savefig('Output/dPy-nmf_end-member_abundances_pies.pdf')

## Examine how well the model reproduces the input samples

How good do the modeled samples compare with the actual samples? One way of examining this is to calculate the similarity (e.g., r2) or dissimilarity (Vmax) of the modeled vs input sample. The plot below 

Let's plot the results as a function of the number of end-member scenarios considered

In [ ]:
plot_width = 8.0
plot_height = 3.0

fig = nmf.plot_child_goodness_of_fit(out_r2=out_r2, out_Vmax=out_Vmax, plot_width=plot_width, plot_height=plot_height);

Not surprisingly, goodness-of-fit between modeled and actal daughter improves as we define more end-members, but the amount of improvement decreases as we add more end-members

In [ ]:
pathlib.Path('Output').mkdir(parents=True, exist_ok=True) # Recursively creates the directory and does not raise an exception if the directory already exists 
fig.savefig('Output/dPy-nmf_modeled_vs_actual_sample_comparison.pdf')

Let's compare the actual daughter with the modeled daughter for each set of end-member scenarios

In [ ]:
x_min = 0
x_max = 3000
fillBetween = True

EMs_min = 1 # Recommended: 1 or 2
EMs_max = 5 # Recommended: Max number of EMs considering

plot_width_multiplier = 4.0
plot_height_multiplier = 2.0

metric_to_plot = 'Vmax' # Options: 'Vmax', 'r2', 'none'

fig = nmf.plot_child_vs_modeled_CDF(labels=labels, EMs_min=EMs_min, EMs_max=EMs_max, xAge=xAge, X=X, Xmodeled=Xmodeled, out_r2=out_r2,
                                    out_Vmax=out_Vmax, x_min=x_min, x_max=x_max, fillBetween=fillBetween, plot_width_multiplier=plot_width_multiplier,
                                   plot_height_multiplier=plot_height_multiplier, metric_to_plot=metric_to_plot);

In [ ]:
pathlib.Path('Output').mkdir(parents=True, exist_ok=True) # Recursively creates the directory and does not raise an exception if the directory already exists 
fig.savefig('Output/dPy-nmf_child_vs_modeled_CDF.pdf')

## Export NMF results as an Excel file

In [ ]:
file_name = 'dPy-nmf_results.xlsx'

nmf.nmf_to_excel(labels=labels, numGrains=numGrains, xAge=xAge,
                 X=X, Wnorm=Wnorm, H=H, modelRecstErr=modelRecstErr, nEMs=nEMs,
                 EMs_min=EMs_min, EMs_max=EMs_max, max_iter=max_iter, nint=nint,
                 tol=tol, in_r2_avg=in_r2_avg, in_r2_range=in_r2_range,
                 in_Vmax_avg=in_Vmax_avg, in_Vmax_range=in_Vmax_range,
                 out_r2=out_r2, out_Vmax=out_Vmax,
                 dist_type=dist_type, bw=bw, bw_x=bw_x, file_name=file_name,
                 version=detritalpy.__version__, verbose=True)

## Plots NMF results as pies on MDS plot

In [ ]:
# Open saved Excel file
df = pd.read_excel('dPy-nmf_results.xlsx',
                      sheet_name='Abundances')
df.set_index('Sample',inplace=True,drop=False)

In [ ]:
metric = False # Set to False for non-metric MDS or set to True for metric MDS (default is False). Non-metric MDS is recommended (Vermeesch, 2013)
criteria = 'Vmax' # Similiarty metric used in the MDS calculation. Options: 'Vmax', 'Dmax', 'R2-PDP', 'R2-KDE' (default is 'Vmax')
n_init = 'metric' # the number of initializations used in the MDS calculation. The final answer will be the initialation that results in the lowest stress value. Set to 'metric' to use metric MDS as initial configuration for non-metric MDS
max_iter = 1000 # The maximum number of iterations the MDS algorthm will run for a given initialization (default = 1000)

x1 = 0 # Lower limit (Ma) of age distribution to conduct MDS analysis on (default = 0)
x2 = 4500 # Upper limit (Ma) of age distribution to conduct MDS analysis on (default = 4500)
xdif = 1 # Interval (Myr) over which distributions are calculated (default = 1)
min_dim = 1 # The minimum number of dimensions over which to calculate MDS (default = 1)
max_dim = 3 # The maximum number of dimensions over which to calculate MDS (default = 3)
dim = 2 # The chosen number of dimensions to plot (default = 2)

model = dFunc.MDS_class(ages, errors, labels, sampleList, metric=metric, criteria=criteria, bw=bw, n_init=n_init, 
                        max_iter=max_iter, x1=x1, x2=x2, xdif=xdif, min_dim=min_dim, max_dim=max_dim, dim=dim)

In [ ]:
figsize = (6,6) # Dimensions of the figure as a tuple (default = (10,10))
savePlot = False # Set to True to create a PDF of the plot in the Output folder (default = True)
fileName = 'shepardPlot.pdf' # Name of the file being saved, if savePlot == True (default = 'shepardPlot.pdf')
plotOneToOneLine = False # Set to True to plot a 1:1 line (default = False)
equalAspect = False # Set to True to display y- and x-axes at the same scale (default = False)

model.shepardPlot(figsize=figsize, savePlot=savePlot, fileName=fileName, plotOneToOneLine=plotOneToOneLine, equalAspect=equalAspect)

In [ ]:
figsize = (12, 12) # Dimensions of the figure as a tuple (default = (10,10))
savePlot = False # Set to True to create a PDF of the plot in the Output folder (default = True)
fileName = 'MDSplot.pdf' # Name of the file being saved, if savePlot == True (default = 'MDSplot.pdf')
plotPie = True # Set to True to plot pie diagrams (default = False)
pieSize = 0.05 # Set to a value (float) to specify pie size (default = 0.05)
agebins = None # Used to define age categories for pie plots (default = None)
pieCategories = ['4EM_EM1','4EM_EM2','4EM_EM3','4EM_EM4'] # Specifiy categories to use in pie diagrams, if pieType == 'Category' (default is None)
agebinsc = [dFunc.colorMe(x) for x in range(len(pieCategories))] # Used to define colors of age categories used for pie plots (default = None)
pieType = 'Category' # Specify type of information to use in plotting pies. Options: 'Age' or 'Category' (default = 'Age')
df = df # Pandas DataFrame that contains pie plot data, if pieType != Age (default = None)
axes = None # array of shape 2,2 with x- and y-axis minimum and maximum values (default = None)
colorBy = 'Default' # specify category to color sample locations by (default = 'Default')
plotLabels = True # set to True to plot sample labels (default = True)
equalAspect = False # set to True to display y- and x-axes at the same scale (default = False)
colors = None # Set to a list of colors of samples or sample groups (optional)

model.MDSplot(figsize=figsize, savePlot=savePlot, fileName=fileName, plotLabels=plotLabels, agebinsc = agebinsc, pieSize=pieSize,
              equalAspect=equalAspect, plotPie = plotPie, pieType=pieType, pieCategories=pieCategories,df=df)